In [1]:
import pandas as pd
import sys
sys.path.append('/home/local/ASURITE/haoyueba/AI4Simulation_SuppluChain')

import os
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from tools.logger import info
from tools import feature_list 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from tools import feature_list
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.cluster import KMeans
import numpy as np

/home/local/ASURITE/haoyueba/.conda/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:


def numerical_features_process(ori_data, numerical_features):
    data = ori_data.copy()
    scaler = StandardScaler()
    data[numerical_features] = scaler.fit_transform(data[numerical_features])
    return data



def date_features_process(ori_data, date_features):
    data = ori_data.copy()
    for feature in date_features:
        data[feature] = pd.to_datetime(data[feature], errors='coerce')
        data[f'{feature}_year'] = data[feature].dt.year.astype(str) + 'y'
        data[f'{feature}_month'] = data[feature].dt.month.astype(str) + 'h'
        data[f'{feature}_day'] = data[feature].dt.day.astype(str) + 'd'
        data[f'{feature}_hour'] = data[feature].dt.hour.astype(str) + 'h'
        data = data.drop(columns=[feature])
    return data


def categorical_features_process(ori_data, categorical_features):
    # 拷贝数据以避免修改原始数据
    data_ori = ori_data.copy()

    # 对所有类别特征统一编码，且处理extra_data中可能不存在的特征
    for feature in categorical_features:
        encoder = LabelEncoder()

        # 检查特征是否存在于 ori_data 和每个 extra_data 中
        combined_feature_data = pd.Series(dtype=str)
        if feature in data_ori.columns:
            combined_feature_data = pd.concat([combined_feature_data, data_ori[feature].astype(str)])


        # 仅当有数据时进行编码
        if not combined_feature_data.empty:
            encoder.fit(combined_feature_data)

            # 在原数据和每个额外数据表中应用相同编码
            if feature in data_ori.columns:
                data_ori[feature] = encoder.transform(data_ori[feature].astype(str))

    # 返回处理后的数据
    return data_ori

def group_features(data, k):
    """
    Discretize features and group them into k groups based on mutual information.

    Args:
        data (DataFrame): Input data with features.
        k (int): The desired number of groups.

    Returns:
        List: Groups of features.
    """
    # Extract specified features
    feature_columns = data.columns  # Replace with the specific columns you want to use
    features = data[feature_columns].copy()

    # Initialize a dictionary to store processed features
    processed_features = {}

    for column in feature_columns:
        # Check the dtype of the specific column
        # print(column)
        # print(features[column])
        if features[column].dtype in [np.float64, np.float32, np.int64, np.int32]:
            # Use KBinsDiscretizer for binning continuous features
            discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
            processed_features[column] = discretizer.fit_transform(features[[column]]).astype(int).flatten()
        else:
            # If the feature is categorical, use LabelEncoder
            encoder = LabelEncoder()
            processed_features[column] = encoder.fit_transform(features[column].astype(str))

    # Convert processed features to a DataFrame
    discretized_features = pd.DataFrame(processed_features)

    # Compute mutual information matrix
    mi_matrix = np.zeros((len(feature_columns), len(feature_columns)))
    feature_array = discretized_features.values
    for i in range(len(feature_columns)):
        mi_matrix[i] = mutual_info_classif(feature_array, feature_array[:, i])

    # Apply K-Means clustering to group features
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(mi_matrix)

    # Group features based on K-Means clustering results
    feature_groups = [[] for _ in range(k)]
    for i, label in enumerate(cluster_labels):
        feature_groups[label].append(feature_columns[i])

    return feature_groups


dataset = 'DataCo'
data = pd.read_csv(f'/home/local/ASURITE/haoyueba/AI4Simulation_SuppluChain/datasets/{dataset}/{dataset}.csv')
# print(data['Product Category Id'])
data =  numerical_features_process(data, feature_list.numerical_features[dataset])
# print(data['Product Category Id'])
data =  date_features_process(data, feature_list.date_features[dataset])
# print(data['Product Category Id'])
data =  categorical_features_process(data, feature_list.categorical_features[dataset])
# print(data['Product Category Id'])


data = data[feature_list.product_info[dataset] + feature_list.order_info[dataset] + feature_list.customer_info[dataset] + \
                        feature_list.shipping_info[dataset]]
# print(data['Product Category Id'])
            


In [12]:
features_groups = group_features(data, 8)

/home/local/ASURITE/haoyueba/.conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/home/local/ASURITE/haoyueba/.conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
/home/local/ASURITE/haoyueba/.conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling exp

In [13]:
features_groups

[['Product Category Id',
  'Category Id',
  'Department Name',
  'Product Card Id',
  'Order Item Cardprod Id'],
 ['Product Price',
  'Order Item Product Price',
  'Sales',
  'Order Item Total',
  'Sales per customer'],
 ['Order Customer Id', 'Customer Id'],
 ['Customer State', 'Customer Zipcode'],
 ['Order Region', 'Market', 'Order Country'],
 ['Order Item Profit Ratio'],
 ['Order Item Quantity',
  'order_date_count',
  'Benefit per order',
  'Order Item Discount',
  'Order Item Discount Rate',
  'Order Profit Per Order',
  'Type',
  'Order Zipcode',
  'Order City',
  'Order State',
  'Customer Segment',
  'order date (DateOrders)_year',
  'order date (DateOrders)_month',
  'order date (DateOrders)_day',
  'order date (DateOrders)_hour'],
 ['Latitude', 'Longitude', 'Customer City', 'Customer Country']]